## 기본 세팅

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
src = cv2.imread('coin.jpg')

# 연습

In [19]:
"""
cv2.cvtColor(image, Convert color type)
cv2.GaussianBlur(image, Kernel range, X axis standard deviation = Alpha{0=Auto})
cv2.canny(gray_image, threshold1, threshold2)
    threshold1 > value = None edge
    threshold2 < value = Perpect Edge
(args1, args2) = cv2.findContour(image, Edge case, expression)
    args1 = Python list
    edge case =  
    
cv2.drawContours(image, contours, contour[idx], color, thickness)
    contour[idx] = -1 (=All)
"""

gray_src = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray_src, (13,13), 0)
cv2.imshow("Source", gray_src)

edged = cv2.Canny(blurred, 40, 140) # Edge detector
cv2.imshow("Edged", edged)

(cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print(f"I count {len(cnts)} coints in this image")

coins = src.copy()
cv2.drawContours(coins, cnts, -1, (0,255,0), 4)
cv2.imshow("Coins", coins)
cv2.waitKey(0)

I count 4 coints in this image


-1

### 크로마키 이미지에서 원본 소스를 빼와서 기존 사진에 합성하기

1. 크로마키 이미지에서 Green채널의 녹색을 제거
   1. 블러 사용 검토(경계부분의 초록색이 남아있음)
2. 1.의 이미지에서 컨투어를 이용해 이미지 잘라내기
3. 2.의 이미지를 배경 이미지에 덞어씌우기
   1. 이미지 resize 및 x,y 좌표 이동 필요

In [55]:
chroma_img = cv2.imread('chroma_key.jpg')
B, G, R = cv2.split(chroma_img)
G = np.where(G > 245, 0, G)
conv_chroma_img = cv2.merge((B,G,R))

In [56]:
cv2.imshow("Chroma Key", conv_chroma_img)
cv2.waitKey(0)

-1

In [57]:
gray_chroma_img = cv2.cvtColor(conv_chroma_img, cv2.COLOR_BGR2GRAY)
# cv2.imshow("gray_chroma_img", gray_chroma_img)

blurred_gray_chroma_img = cv2.GaussianBlur(gray_chroma_img, (15,15), 0)
# cv2.imshow("blurred_gray_chroma_img", blurred_gray_chroma_img)

edged_blurred_gray_chroma_img = cv2.Canny(blurred_gray_chroma_img, 20, 200)
cv2.imshow("edged_blurred_gray_chroma_img", edged_blurred_gray_chroma_img)
cv2.waitKey(0)

(contours, _) = cv2.findContours(edged_blurred_gray_chroma_img.copy(), cv2.RETR_EXTERNAL,\
                cv2.CHAIN_APPROX_SIMPLE)

print(f"Find contour count : {len(contours)}")


Find contour count : 2


In [38]:
for (idx, contour) in enumerate(contours):
    (x, y, w, h) = cv2.boundingRect(contour)
    
    print(f"Contour # {idx+1}")
    print(F"x : {x} / y : {y} / w : {w} / h : {h} ")
    contour_temp = conv_chroma_img[y : y+h, x : x+w]
    print(f"Contour_temp shape : {contour_temp.shape}")
    print()
    # cv2.imshow("Contour",contour_temp)
    # cv2.waitKey(0)
    
    mask = np.zeros(chroma_img.shape[:2], dtype='uint8')
    print(f"Mask shape : {mask.shape}")
    ((centerX, centerY), radius) = cv2.minEnclosingCircle(contour)
    cv2.circle(mask, (int(centerX), int(centerY)), int(radius), 255, -1)
    mask = mask[y : y+h, x : x+w]

    contour_img = cv2.bitwise_and(contour_temp, contour_temp, mask=mask)
    cv2.imshow('Masked Contour', contour_img)
    cv2.waitKey(0)

Contour # 1
x : 694 / y : 419 / w : 38 / h : 74 
Contour_temp shape : (74, 38, 3)

Mask shape : (720, 1280)
Contour # 2
x : 552 / y : 248 / w : 526 / h : 472 
Contour_temp shape : (472, 526, 3)

Mask shape : (720, 1280)


In [63]:
# img1, img2 입력, 이미지 확인
img = cv2.imread('chroma_key.jpg')
background_img = cv2.imread('busstop.jpg')
cv2.imshow("Original", img)
cv2.imshow("Background", background_img)
cv2.waitKey()
cv2.destroyAllWindows()

In [74]:
# 이미지 색을 RGB -> HSV로 변경 후 Green 기준으로 amsk 생성
img2hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(img2hsv, (50,150,0), (70,255,255))
mask_inv = cv2.bitwise_not(mask)

cv2.imshow("mask", mask)
cv2.imshow("mask_inv", mask_inv)
cv2.waitKey()
cv2.destroyAllWindows()

In [76]:
# 마스크 영역만 추출
img2_fg = cv2.bitwise_and(img, img, mask=mask_inv)

In [79]:
# 크로마키 이미지의 높이, 너비, 채널 구하기
height, width, channels = img.shape
# 배경 이미지에서 크로마키 마스크 영역을 관심영역으로 설정
roi = background_img[0:height, 0:width]

In [80]:
print(roi.shape, img2_fg.shape)

(720, 1200, 3) (720, 1280, 3)


In [ ]:
dst = cv2.addWeighted(roi, 1, img2_fg, 1, 0)
new_img = background_img.copy()
new_img[0:height, 0:weight] = dst

cv2.imshow("Image",new_img)
cv2.waitKey()
cv2.destroyAllWindows()